<a href="https://colab.research.google.com/github/JayThibs/hyperdrive-vs-automl-plus-deployment/blob/main/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning using HyperDrive

# Environment and Import Dependencies

Here we specify the conda dependencies.

In [1]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
    - azureml-defaults
    - scikit-learn

Overwriting conda_dependencies.yml


In [2]:
from azureml.core import Environment

# Creating a conda environment for model training. It needs to be included in ScriptRunConfig.

sklearn_env = Environment.from_conda_specification(name='sklearn_env', file_path='./conda_dependencies.yml')

In [3]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import logging
import os
import csv

from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


# Write Training File for our Hyperdrive Model

In [4]:
%%writefile train.py

import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

import argparse
from sklearn.metrics import mean_squared_error
import joblib
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
import datetime

# Data Exploration was done in a notebook beforehand

# def dates(X):
#     """
#     date_recorded: this might be a useful variable for this analysis, although the year itself would be 
#     useless in a practical scenario moving into the future. We will convert this column into a datetime, 
#     and we will also create 'year_recorded' and 'month_recorded' columns just in case those levels prove 
#     to be useful. A visual inspection of both casts significant doubt on that possibility, but we'll proceed 
#     for now. We will delete date_recorded itself, since random forest cannot accept datetime
#     """
#     for i in [X]:
#         i['date_recorded'] = pd.to_datetime(i['date_recorded'])
#         i['year_recorded'] = i['date_recorded'].apply(lambda x: x.year)
#         i['month_recorded'] = i['date_recorded'].apply(lambda x: x.month)
#         i['date_recorded'] = (pd.to_datetime(i['date_recorded'])).apply(lambda x: x.toordinal())
#     return X


# def date_parser(df):
#     date_recorder = list(map(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d'),
#                              df['date_recorded'].values))
#     df['year_recorder'] = list(map(lambda x: int(x.strftime('%Y')), date_recorder))
#     df['weekday_recorder'] = list(map(lambda x: int(x.strftime('%w')), date_recorder))
#     df['yearly_week_recorder'] = list(map(lambda x: int(x.strftime('%W')), date_recorder))
#     df['month_recorder'] = list(map(lambda x: int(x.strftime('%m')), date_recorder))
#     df['age'] = df['year_recorder'].values - df['construction_year'].values
#     del df['date_recorded']
#     return df


def bools(X):
    """
    public_meeting: we will fill the nulls as 'False'
    permit: we will fill the nulls as 'False'
    """
    z = ['public_meeting', 'permit']
    for i in z:
        X[i].fillna(False, inplace = True)
        X[i] = X[i].apply(lambda x: float(x))
    return X


def small_n(X):
    "Collapsing small categorical value counts into 'other'"
    cols = [i for i in X.columns if type(X[i].iloc[0]) == str]
    X[cols] = X[cols].where(X[cols].apply(lambda x: x.map(x.value_counts())) > 100, "other")
    return X



# We loaded the dataset into Azure and we are grabbing it here.

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

# download config file in azure and put it in the current Notebooks folder
ws = Workspace.from_config()

dataset = Dataset.get_by_name(ws, name='Pump-it-Up-dataset')
df = dataset.to_pandas_dataframe()
y = df['status_group']
del df['status_group']
X = df

# A lot of null values for construction year. Of course, this is a missing value (a placeholder).
# For modeling purposes, this is actually fine, but we'll have trouble with visualizations if we
# compare the results for different years, so we'll set the value to something closer to
# the other values that aren't placeholders. Let's look at the unique years and set the null
# values to 50 years sooner.
# Alright, let's set it to 1910
X.loc[X['construction_year'] < 1950, 'construction_year'] = 1910

# Cleaning up the features of our dataset
# X = dates(X)
# x = date_parser(X)
X = bools(X)
X['population'] = np.log(X['population'])
X = small_n(X)

# Splitting the dataset into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

run = Run.get_context()


def main():
    # Adds arguments to script
    parser = argparse.ArgumentParser()

    # Setting the hyperparameters we will be optimizing for you Random Forest model
    parser.add_argument('--max_depth', type=int, default=5, help='The maximum depth of the trees.')
    parser.add_argument('--min_samples_split', type=int, default=4, help='The minimum number of samples required to split an internal node.')
    parser.add_argument('--n_estimators', type=int, default=750, help='The number of trees in the forest.')

    args = parser.parse_args()

    run.log("Max depth of the trees:", np.int(args.max_depth))
    run.log("Minimum number of samples required to split:", np.int(args.min_samples_split))
    run.log("Number of trees:", np.int(args.n_estimators))

    # Fitting a Random Forest model to our data. 
    # Sidenote: I also tried XGBoost on my local machine, but it did not perform as well.
    # RF has a score of 0.811, XGBoost has a score of 0.745
    # Since I did not use a validation set, it's possible that I'm just overfitting with RF.
    # But I wanted to focus on the end-to-end process for this project so I didn't bother with 
    # a validation set.
    rf = RandomForestClassifier(max_depth=args.max_depth,
                                min_samples_split=args.min_samples_split,
                                n_estimators=args.n_estimators,
                                criterion='gini',
                                oob_score=True,
                                random_state=42,
                                n_jobs=-1).fit(X_train, y_train)
    
    # Predicting on the test set
    predictions = rf.predict(X_test)
    pred = pd.DataFrame(predictions, columns = [y_test.columns[0]])

    # Calculate recall to test how well we do on True Positives
    # We can imagine a real scenario where we want to build a model
    # that does not miss the non-functioning water pumps, and we
    # care much less functioning water pumps that are incorrectly
    # predicted as non-functional. 
    recall = recall_score(y_test, pred, average='micro')
    run.log("Recall", np.float(recall))

    os.makedirs('outputs', exist_ok=True)
    joblib.dump(rf, 'outputs/rf_model.pkl')

if __name__ == '__main__':
    main()

Overwriting train.py


# Dataset

Getting our data and initialize a workspace object from persisted configuration. We placed the config file in .\config.json.

In [29]:
from azureml.core import Workspace, Experiment

# download config file in azure and put it in the current Notebooks folder
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Pump-it-Up-Data-Mining-the-Water-Table")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142186
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-142186


In [30]:
# We loaded the dataset into Azure and we are grabbing it here.
from azureml.core import Dataset

# If you get an error during training and try to rerun this cell, it may get stuck in execution.
# I'm not sure what's the issue, but it gets resolved by stopping compute before rerunning the notebook.
dataset = Dataset.get_by_name(ws, name='Pump-it-Up-dataset')
df = dataset.to_pandas_dataframe()
y = df['status_group']
del df['status_group']
df.describe()

KeyboardInterrupt: 

In [13]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [26]:
df.columns[df.dtypes==object]

Index(['funder', 'installer', 'basin', 'lga', 'ward', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_class', 'management', 'management_group',
       'payment_type', 'water_quality', 'quantity_group', 'source',
       'source_class', 'waterpoint_type'],
      dtype='object')

In [25]:
df.columns

Index(['date_recorded', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_class',
       'management', 'management_group', 'payment_type', 'water_quality',
       'quantity_group', 'source', 'source_class', 'waterpoint_type'],
      dtype='object')

In [24]:
columns_to_remove = ['id','amount_tsh',  'num_private', 'region', 
          'quantity', 'quality_group', 'source_type', 'payment', 
          'waterpoint_type_group',
         'extraction_type_group']
df = df.drop('id', axis=1)
df.head()

KeyError: "['id'] not found in axis"

# Setting up our Compute Target

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Creating a compute cluster if there isn't one that is already created.

cpu_cluster_name = 'hypr-auto-clustr'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new computer target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_v2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
# Using get_status() to get a detailed status for the current compute cluster.
print(cpu_cluster.get_status().serialize())

{'currentNodeCount': 3, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 3, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-04-07T18:02:48.072000+00:00', 'errors': None, 'creationTime': '2021-04-07T16:46:46.252734+00:00', 'modifiedTime': '2021-04-07T16:47:01.701608+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [9]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

notebook142186 ComputeInstance Succeeded
hypr-auto-clustr AmlCompute Succeeded


In [10]:
cpu_cluster

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-142186', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-142186'), name=hypr-auto-clustr, id=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourceGroups/aml-quickstarts-142186/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-142186/computes/hypr-auto-clustr, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags=None)

# Hyperdrive Configuration

We are using Random Forest to train our model. We will use Hyperdrive to do a hyperparameter search to optimize our model.

We will be using Random Sampling since it is more efficient than grid search for finding an optima.



In [11]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import quniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specifying parameter sampler.
# - Here we use Bayesian Hyperparameter Sampling to search the hyperparameter space for the best model.
# - Essentially, Bayesian Sampling builds a probability model of the objective function we are trying 
#   to minimize and uses it to select the most promising hyperparameters to evaluate in the true objective function.
# - For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or 
#   equal to 20 times the number of hyperparameters being tuned. Recommendend value:60.
#   We will be optimizing 3 hyperparameters for this project, therefore we choose 60 max_total_runs.
# - We also use quniform to search the hyperparameter space since quniform(low, high, q) creates uniform distriution 
#   between low and high values, separated by spacing q.

ps = BayesianParameterSampling(
    {
    'max_depth': quniform(3, 15, 1), # Maximum depth of the trees
    'min_samples_split': choice(2, 4, 6), # Minimum number of samples required to split
    'n_estimators' : quniform(500, 1000, 50) # Number of trees
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Creating a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory=os.path.join('./'),
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Creating a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    primary_metric_name='norm_macro_recall',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=60,
                                    max_concurrent_runs=4)

# Submit Experiment and Run Details

In [12]:
# Submitting a HyperDrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2e33f43b-0592-4de2-aaf6-92407a66352b
Web View: https://ml.azure.com/experiments/Pump-it-Up-Data-Mining-the-Water-Table/runs/HD_2e33f43b-0592-4de2-aaf6-92407a66352b?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-142186/workspaces/quick-starts-ws-142186

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-07T18:04:25.745615][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-07T18:04:25.899457][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-07T18:04:26.2532056Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-04-07T18:04:25.176901][API][INFO]Experiment created<END>\n"

Execution Summary
RunId: HD_2e33f43b-0592-4de2-aaf6-92407a66352b
Web View: https://ml.azure.com/experiments/Pump-it-Up-Data-Mining-the-Water-Table/runs/HD_2e33f43b-0592-4d

{'runId': 'HD_2e33f43b-0592-4de2-aaf6-92407a66352b',
 'target': 'hypr-auto-clustr',
 'status': 'Canceled',
 'startTimeUtc': '2021-04-07T18:04:24.446612Z',
 'endTimeUtc': '2021-04-07T18:29:38.667094Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name"

# Saving our Best Random Forest / Hyperdrive Model

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run:', best_run)
print('Metrics:', best_run_metrics['recall_score_micro'])

hyperdrive_model = best_run.register_model(model_name="rf_hyperdrive_model", model_path="./outputs/model.pkl")